In [1]:
from pathlib import Path
CUR_ABS_DIR = Path.cwd().resolve()
PROJ_DIR = (CUR_ABS_DIR / '../../../').resolve()
print(PROJ_DIR)

/mnt/832acd65-7396-480e-aa76-dca6765861b0/research-repo/projects/2025.09_hydrology_Switzerland/codes/swiss-river-network-benchmark


In [2]:
# # Magic home
# %cd /home/benjamin/git/swiss-river-network/

import numpy as np
import matplotlib.pyplot as plt


from swissrivernetwork.benchmark.dataset import read_graph, read_csv_test, read_csv_train
from swissrivernetwork.util.datetime import to_human, from_unix_days

# Labels
label = {}
label['swiss-1990'] = 'Swiss 1990'
label['swiss-2010'] = 'Swiss 2010'
label['zurich'] = 'Zurich'

# Variables
GRAPH_NAMES = ['swiss-1990', 'swiss-2010', 'zurich']

def print_dates(df):
    vmin, vmax = df['epoch_day'].min(), df['epoch_day'].max()
    print('Training:', to_human(from_unix_days(vmin)), to_human(from_unix_days(vmax)))

for graph_name in GRAPH_NAMES:
    # Load Graph Data:
    g,e = read_graph(graph_name, base_dir=PROJ_DIR)
    n = g.shape[0]

    # compute covered area:
    dx = g[:, 0].max() - g[:, 0].min()
    dy = g[:, 1].max() - g[:, 1].min()
    dx, dy = dx / 1000, dy / 1000 # in km
    area = dx*dy
    area_per_n = area/n

    # print table values:
    print(f'~~~ {label[graph_name]} ~~~ ')
    print('Stations:', n)
    print('Density:', area_per_n)

    # Get the years
    df = read_csv_train(graph_name, base_dir=PROJ_DIR)
    print_dates(df)
    df = read_csv_test(graph_name, base_dir=PROJ_DIR)
    print_dates(df)

~~~ Swiss 1990 ~~~ 
Stations: 28
Density: tensor(1596.2872)
Training: 1990-01-02 00:00:00+00:00 2012-12-31 00:00:00+00:00
Training: 2013-01-02 00:00:00+00:00 2020-12-31 00:00:00+00:00
~~~ Swiss 2010 ~~~ 
Stations: 63
Density: tensor(1039.8373)
Training: 2005-01-02 00:00:00+00:00 2017-12-31 00:00:00+00:00
Training: 2018-01-01 00:00:00+00:00 2020-12-31 00:00:00+00:00
~~~ Zurich ~~~ 
Stations: 15
Density: tensor(73.8142)
Training: 2009-01-01 00:00:00+00:00 2019-12-31 00:00:00+00:00
Training: 2020-01-01 00:00:00+00:00 2022-10-31 00:00:00+00:00
